# Understanding Image Augmentation

You can look into my other notebooks to get more detailed augmentation insights.
list of my other notebooks - 

1. Understanding Image Augmentation - [link](https://www.kaggle.com/amritpal333/understanding-image-augmentation-ranzcr-comp)
2. CLAHE augmentation  - [link](https://www.kaggle.com/amritpal333/clahe-augmentation-ranzcr-comp)
3. Findings your own mean,standerd deviation for the dataset - [link](https://www.kaggle.com/amritpal333/using-custom-mean-std-ranzcr-comp)


## Upvote the notebooks if you find them insightful.

In [ ]:
#!pip install albumentations
import albumentations


import torch
from albumentations import ( Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, IAAAdditiveGaussianNoise, Transpose, ToGray )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import matplotlib.pyplot as plt

seed = 42

import pandas as pd
import os
import cv2
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm



class Ranzcr_jpg_train_dataset(Dataset):    
	
    def __init__(self, files_folder_path, df, num_channels , transfroms = None ):
		self.files_folder_path = files_folder_path
		self.df = df
		self.transforms = transfroms
		self.num_channels = num_channels

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):

		image_id = self.df.StudyInstanceUID.values[idx]
		if self.num_channels == 1:
			image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ), 0)
        
		else:
			image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ))
			image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image #*(1/255)
        
		if self.transforms:
			image = self.transforms(image=image)['image']
		labels = self.df[self.df.StudyInstanceUID == image_id].values.tolist()[0][1:-1]
		labels = torch.tensor(labels,dtype= torch.float32) #.view(1,-1)

		return image, labels

# defining your own dataset

Rather than looking into images from folder at random, i want to look at the images as they are sent to the model.
this removes any scope of error.

In [ ]:
class Ranzcr_jpg_train_dataset(Dataset):    
	def __init__(self, files_folder_path, df, num_channels , transfroms = None ):
		self.files_folder_path = files_folder_path
		self.df = df
		self.transforms = transfroms
		self.num_channels = num_channels

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):

		image_id = self.df.StudyInstanceUID.values[idx]
		image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ))
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		image = image*(1/255)
        
		if self.transforms:
			image = self.transforms(image=image)['image']
		labels = self.df[self.df.StudyInstanceUID == image_id].values.tolist()[0][1:-1]
		labels = torch.tensor(labels,dtype= torch.float32) #.view(1,-1)

		return image, labels

In [ ]:
train_path = '../input/ranzcr-clip-catheter-line-classification/train'
train_files = os.listdir(train_path)

train_df = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')

train = train_df.reset_index(drop=True) # reset index on both dataframes

print(train.shape)

num_channel = 3

img_size = 255

# Baseline

In [ ]:
class Ranzcr_jpg_train_dataset(Dataset):    
	def __init__(self, files_folder_path, df, num_channels , transfroms = None ):
		self.files_folder_path = files_folder_path
		self.df = df
		self.transforms = transfroms
		self.num_channels = num_channels

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):

		image_id = self.df.StudyInstanceUID.values[idx]
		image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ))
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		#image = image*(1/recale_image_by)
        
		if self.transforms:
			image = self.transforms(image=image)['image']
		labels = self.df[self.df.StudyInstanceUID == image_id].values.tolist()[0][1:-1]
		labels = torch.tensor(labels,dtype= torch.float32) #.view(1,-1)

		return image, labels

In [ ]:

def Show_Xrays(augmentation):
    num_channels = 3
    trainset = Ranzcr_jpg_train_dataset(train_path,  train, num_channels , augmentation )
    trainloader = DataLoader(trainset, batch_size = 16 , num_workers = 3 , shuffle = True)

    fig = plt.figure()
    fig.set_size_inches(25, 25)
    #fig.savefig('test2png.png', dpi=100)

    for batch_i, (data, target) in tqdm(enumerate(trainloader)):
        #print(data.shape)
        if batch_i == 3:
            break
        for i in range(data.shape[0]):
            ax = plt.subplot(4,4, i+1)
            plt.tight_layout()
            ax.axis('off')
            plt.imshow(data[i])
            

In [ ]:
mean_list = [[ 0.4914168 , 0.4914168 , 0.4914168] , [0.485,0.456,0.406] , [0.9,0.9,0.9] , 0.496 ]
std_list = [[0.407278, 0.407278 , 0.407278] , [0.229,0.224,0.225] , [0.9,0.9,0.9] , 0.407278]

mean = mean_list[0]
std=  std_list[0]

print(mean , std)

In [ ]:
train_augs = albumentations.Compose([albumentations.Resize(height=img_size, width=img_size, p=1.0), 
                                     albumentations.Normalize(mean= mean ,std= std ,),
                                    ])

Show_Xrays(train_augs)

### Findings- 
If we are training from scratch, we need to find a new mean,std.
If using pretrained weights, then most likely stick to the values on pretraining dataset.

Notebook on how to find your own mean and std deviation ? https://www.kaggle.com/amritpal333/using-custom-mean-std-ranzcr-comp

In [ ]:
train_augs = albumentations.Compose([   albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1),
                                        albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7),                            
                                    ])
Show_Xrays(train_augs)

In [ ]:
train_augs = albumentations.Compose([   albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1),
                                        albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7), 
                                        albumentations.CLAHE(clip_limit=(1,10), p= 1),
                                        #albumentations.Normalize(mean= mean ,std= std ,)
                                    ])
Show_Xrays(train_augs)

###  Findings- CLAHE is awesome
CLAHE works like magic to improve the visualisation of the catheters.
You can look into my detailed analysis on how CLAHE effects xrays in this competition in my notebook -

### [link](https://www.kaggle.com/amritpal333/clahe-augmentation-ranzcr-comp)

# Normalizing data first

In [ ]:
train_augs = albumentations.Compose([   albumentations.Normalize(mean= mean ,std= std ,) , 
                                         albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1),
                                        albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7), 
                                        #albumentations.CLAHE(clip_limit=(1,10), p= 1)
                                        
                                    ])
Show_Xrays(train_augs)

### findings - Normalizing images before using other augmentation works only in simple cases, and raises an error in others.

e.g. using CLAHE on normalized data
>     raise TypeError("clahe supports only uint8 inputs")
>     TypeError: clahe supports only uint8 inputs

So, use Normalize as the last in the list of your augmentations.

# Trying differnt combinations of image augmentation

(I hope to find one that magically works!)

In [ ]:
train_augs = albumentations.Compose([albumentations.Resize(img_size, img_size),
                albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1), 
				albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=1),
				albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=1),
				albumentations.CLAHE(clip_limit=(1,4), p=1),
                #albumentations.imgaug.transforms.IAASharpen(alpha=(0.2, 0.3), lightness=(0.5, 0.7), p=1),
                #albumentations.Cutout(max_h_size=int(img_size * 0.05), max_w_size=int(img_size * 0.05), num_holes=5, p= 0.5),
				#albumentations.Normalize(mean= mean ,  std= std ,) 
               ])

Show_Xrays(train_augs)



In [ ]:
train_augs = albumentations.Compose([albumentations.Resize(img_size, img_size),
                albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1), 
				#albumentations.ShiftScaleRotate(shift_limit_x=(-0.0125, 0.0125),shift_limit_y=(-0.0125, 0.0125) ,rotate_limit=(-15, 15) , p=1),
				albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=1),
				albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=1),
				albumentations.CLAHE(clip_limit=(1,4), p=1),
                ########albumentations.OpticalDistortion(distort_limit=1.0),
				#####albumentations.ElasticTransform(alpha=3),
                #albumentations.GaussNoise(var_limit=[10, 50], p=1),
                #albumentations.MotionBlur(p=1),
                #albumentations.MedianBlur(p=1),
                #albumentations.augmentations.transforms.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1),
                albumentations.imgaug.transforms.IAASharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1),
                albumentations.imgaug.transforms.IAAEmboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=1),
                ###albumentations.imgaug.transforms.IAAPerspective (scale=(0.05, 0.1), keep_size=True, p=1),
                albumentations.augmentations.transforms.ToGray(p=1),
                #albumentations.augmentations.transforms.RandomGamma(gamma_limit=(80, 120), eps=None, p=1),
                #albumentations.Cutout(max_h_size=int(img_size * 0.05), max_w_size=int(img_size * 0.05), num_holes=5, p= 0.5),
				#albumentations.Normalize(mean= mean ,  std= std ,) 
               ])

Show_Xrays(train_augs)



# Saving all varints of augmentation to folder

to download later on

https://albumentations.ai/docs/api_reference/augmentations/transforms/#albumentations.augmentations.transforms.RandomRain

In [ ]:
augment_list = [albumentations.Resize(img_size, img_size),
                albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1), 
				albumentations.ShiftScaleRotate(p=1),
				albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=1),
				albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=1),
				albumentations.CLAHE(clip_limit=(1,4), p=1),
                albumentations.OpticalDistortion(distort_limit=1.0),
				albumentations.ElasticTransform(alpha=3),
                albumentations.GaussNoise(var_limit=[10, 50], p=1),
                albumentations.MotionBlur(p=1),
                albumentations.MedianBlur(p=1),
                albumentations.augmentations.transforms.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1),
                albumentations.imgaug.transforms.IAASharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1),
                albumentations.imgaug.transforms.IAAEmboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=1),
                albumentations.imgaug.transforms.IAAPerspective (scale=(0.05, 0.1), keep_size=True, p=1),
                albumentations.augmentations.transforms.ToGray(p=1),
                albumentations.augmentations.transforms.RandomGamma(gamma_limit=(80, 120), eps=None, p=1),
                albumentations.Cutout(max_h_size=int(img_size * 0.05), max_w_size=int(img_size * 0.05), num_holes=10, p= 0.5),
				albumentations.Normalize(mean= mean ,  std= std ,) 
               ]

make a smaller version of list , the ones you are interested in.

In [ ]:
augment_list = [  albumentations.Resize(img_size, img_size),
                albumentations.RandomResizedCrop(img_size, img_size, scale=(0.9, 1), p=1), 
				  ########albumentations.HorizontalFlip(p=1),
				  albumentations.ShiftScaleRotate(p=1),
				   albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=1),
				   albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=1),
				   albumentations.CLAHE(clip_limit=(1,4), p=1),
                   albumentations.OpticalDistortion(distort_limit=1.0),
				   #albumentations.GridDistortion(num_steps=5, distort_limit=1.),
				   albumentations.ElasticTransform(alpha=3),
                 albumentations.GaussNoise(var_limit=[10, 50], p=1),
                 #albumentations.GaussianBlur(p=1),
                 albumentations.MotionBlur(p=1),
                albumentations.MedianBlur(p=1),
                albumentations.augmentations.transforms.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1),
                #albumentations.imgaug.transforms.IAASuperpixels(p_replace=0.1, n_segments=100, p=1),
                albumentations.imgaug.transforms.IAASharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1),
                albumentations.imgaug.transforms.IAAEmboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=1),
                albumentations.imgaug.transforms.IAAPerspective (scale=(0.05, 0.1), keep_size=True, p=1),
                ######albumentations.augmentations.domain_adaptation.HistogramMatching (reference_images , blend_ratio=(0.5, 1.0), p=0.5),
                #albumentations.augmentations.transforms.ToSepia(p=1),
                albumentations.augmentations.transforms.ToGray(p=1),
                #albumentations.imgaug.transforms.IAAAdditiveGaussianNoise(loc=0, scale=(2.5500000000000003, 12.75), per_channel=False, p=1),
                
                albumentations.augmentations.transforms.RandomGamma(gamma_limit=(80, 120), eps=None, p=1),
                #albumentations.augmentations.transforms.Solarize(threshold=128, p=1),
                #albumentations.augmentations.transforms.RandomFog(fog_coef_lower=0.3, fog_coef_upper=1, alpha_coef=0.08, p=1),
                #albumentations.augmentations.transforms.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, drop_color=(200, 200, 200), blur_value=7, brightness_coefficient=0.7, rain_type=None, p=1),
				#albumentations.Cutout(max_h_size=int(img_size * 0.1), max_w_size=int(img_size * 0.1), num_holes=5, p=1),
				albumentations.Normalize(mean= mean ,  std= std ,) 
               ]

In [ ]:
def Show_save_Xrays(augmentation , name):
    num_channels = 3
    trainset = Ranzcr_jpg_train_dataset(train_path,  train, num_channels , augmentation )
    trainloader = DataLoader(trainset, batch_size = 1 , num_workers = 3 , shuffle = False)

    fig = plt.figure()
    fig.set_size_inches(25, 25)
    #fig.savefig('test2png.png', dpi=100)

    for batch_i, (data, target) in tqdm(enumerate(trainloader)):
        #print(data.shape)
        if batch_i == 1:
            break
        for i in range(data.shape[0]):
            ax = plt.subplot(1 ,1, i+1)
            plt.tight_layout()
            ax.axis('off')
            plt.imshow(data[i])
            fig.savefig( str(name) + '.png', dpi=200)

In [ ]:
Save_to_folder = False


if Save_to_folder == True:
    for i in range(len(augment_list)):
        train_augs = albumentations.Compose([ albumentations.Resize(img_size, img_size) , augment_list[i]])
        Show_save_Xrays(train_augs , augment_list[i])

In [ ]:
!zip -r -q 'Albumentation_1*1.zip'  ./

In [ ]:
!rm ./*.zip

In [ ]:
!rm ./*.png